In [1]:
import argparse
import h5py
import os

In [2]:
keep = ['class_name', 'name', 'batch_input_shape', 'input_dim', 'output_dim', 'dtype', 'go_backwards', 'units', 'activation', 'recurrent_activation', 'use_bias']
ignore = ['RandomUniform', 'GlorotUniform', 'Orthogonal', 'Zeros']

model_path = 'lstm.h5'
filename = os.path.basename(model_path)
filename = filename[:filename.rfind('.')] + '.model'

In [3]:
model_str = ''

with h5py.File(model_path, 'r') as h5_file:
    temp = []
    keys = []
    h5_file.visit(temp.append)

    for key in temp:
        if 'model_weights' in key and ('embeddings' in key or 'bias' in key or 'kernel' in key):
            keys.append(key)

    config = h5_file.attrs.get('model_config')
    num_tabs = -1
    phase = ''
    get_str = ''
    ignore_count = 0
    in_list = 0
    in_layer = False
    key = ''
    name = ''
    stored_name = ''
    stored_layer_name = ''
    layer_name = ''
    attrs = {}

    for index, char in enumerate(config):
        if char == '{':
            num_tabs += 1
        elif char == '}':
            num_tabs -= 1
        elif char == '"':
            if phase == '':
                phase = 'get'
            else:
                if config[index + 1] == ':':
                    if get_str in keep:
                        key = get_str
                        ignore_count = 2
                    else:
                        phase = ''

                    get_str = ''
        elif ignore_count != 0:
            ignore_count -= 1
        elif phase == 'get':
            get_str += char

            if char == '[':
                in_list += 1
            elif char == ']':
                in_list -= 1

        if key != '' and (in_list == 0 and config[index + 1] in ',}'):
            if get_str not in ignore:
                if get_str == 'Sequential':
                    model_str += get_str + '\n'
                elif get_str == layer_name or get_str == stored_layer_name or 'sequential' in get_str:
                    pass
                elif key in ('class_name', 'name'):
                    name += get_str

                    if key == 'class_name':
                        if in_layer:
                            for k in keys:
                                if layer_name + '/' in k:
                                    if 'embeddings' in k or 'kernel' in k:
                                        weights = list(h5_file[k])
                                        trimmed = []

                                        for weight in weights:
                                            trimmed.append(list(weight))
                                    attrs['embeddings' if 'embeddings' in k else ('bias' if 'bias' in k else ('recurrent_kernel' if 'recurrent_kernel' in k else 'kernel'))] = str(trimmed) if 'embeddings' in k or 'kernel' in k else str(list(h5_file[k]))

                            if 'bidirectional' not in layer_name:
                                for k, value in attrs.items():
                                    model_str += '  ' + k + ': ' + value + '\n'

                            attrs = {}

                        in_layer = True
                    elif key == 'name':
                        layer_name = get_str

                    if ' ' in name:
                        if stored_name != '':
                            name = name.split(' ')
                            stored_name = stored_name.split(' ')
                            model_str += ' '.join([stored_name[0], name[0], stored_name[1], name[1]]) + '\n'
                            stored_name = ''
                        elif 'Bidirectional' not in name:
                            model_str += name + '\n'
                        else:
                            stored_name = name
                            stored_layer_name = layer_name

                        name = ''
                    else:
                        name += ' '
                else:
                    attrs[key] = get_str

            phase = ''
            key = ''
            get_str = ''

        if get_str != '' and (in_list == 0 and config[index + 1] in ',}'):
            get_str = ''
            phase = ''

    for k in keys:
        if layer_name + '/' in k:
            if 'embeddings' in k or 'kernel' in k:
                weights = list(h5_file[k])
                trimmed = []

                for weight in weights:
                    trimmed.append(list(weight))

            attrs['embeddings' if 'embeddings' in k else ('bias' if 'bias' in k else ('recurrent_kernel' if 'recurrent_kernel' in k else 'kernel'))] = str(trimmed) if 'embeddings' in k or 'kernel' in k else str(list(h5_file[k]))

    for k, value in attrs.items():
        model_str += '  ' + k + ': ' + value + '\n'

In [4]:
with open(filename, 'w') as model_file:
    model_file.write(model_str)